In [1]:
!pip install --upgrade openai


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
def generate_functions(labels: dict) -> list:
    return [
        {   
            "type": "function",
            "function": {
                "name": "enrich_entities",
                "description": "Enrich Text with Knowledge Base Links",
                "parameters": {
                    "type": "object",
                        "properties": {
                            "r'^(?:' + '|'.join({labels}) + ')$'": 
                            {
                                "type": "array",
                                "items": {
                                    "type": "string"
                                }
                            }
                        },
                        "additionalProperties": False
                },
            }
        }
    ]

In [3]:
def assisstant_message():
    return f"""
EXAMPLE:
    Text: 'In Germany, in 1440, goldsmith Johannes Gutenberg invented the movable-type printing press. His work led to an information revolution and the unprecedented mass-spread / 
    of literature throughout Europe. Modelled on the design of the existing screw presses, a single Renaissance movable-type printing press could produce up to 3,600 pages per workday.'
    {{
        "gpe": ["Germany", "Europe"],
        "date": ["1440"],
        "person": ["Johannes Gutenberg"],
        "product": ["movable-type printing press"],
        "event": ["Renaissance"],
        "quantity": ["3,600 pages"],
        "time": ["workday"]
    }}
--"""

In [4]:
def user_message(text):
    return f"""
TASK:
    Text: {text}
"""

In [5]:
def system_message(labels):
    return f"""
You are an expert in Natural Language Processing. Your task is to identify common Named Entities (NER) in a given text.
The possible common Named Entities (NER) types are exclusively: ({", ".join(labels)})."""

In [6]:
labels = [
    "person",      # people, including fictional characters
    "fac",         # buildings, airports, highways, bridges
    "org",         # organizations, companies, agencies, institutions
    "gpe",         # geopolitical entities like countries, cities, states
    "loc",         # non-gpe locations
    "product",     # vehicles, foods, appareal, appliances, software, toys 
    "event",       # named sports, scientific milestones, historical events
    "work_of_art", # titles of books, songs, movies
    "law",         # named laws, acts, or legislations
    "language",    # any named language
    "date",        # absolute or relative dates or periods
    "time",        # time units smaller than a day
    "percent",     # percentage (e.g., "twenty percent", "18%")
    "money",       # monetary values, including unit
    "quantity",    # measurements, e.g., weight or distance
]

In [ ]:
text = """The Beatles were an English rock band formed in Liverpool in 1960, comprising John Lennon, Paul McCartney, George Harrison, and Ringo Starr."""

In [7]:
from openai import OpenAI
import os
# client = OpenAI()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

messages = [
      {"role": "system", "content": system_message(labels=labels)},
      {"role": "assistant", "content": assisstant_message()},
      {"role": "user", "content": user_message(text=text)}
  ]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=generate_functions(labels),
    tool_choice={"type": "function", "function" : {"name": "enrich_entities"}}, 
    temperature=0,
    frequency_penalty=0,
    presence_penalty=0,
)

response_message = response.choices[0].message

In [8]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_im0b7xoKKccOvGeHkQNl8lVx', function=Function(arguments='{"person":["John Lennon","Paul McCartney","George Harrison","Ringo Starr"],"org":["The Beatles"],"gpe":["Liverpool","England"],"date":["1960"]}', name='enrich_entities'), type='function')])